In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, mape, r2

# Create and compile the LSTM model, adding L2 regularization and adjusting the structure
def build_lstm_model(input_shape, output_shape):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.001))),
        Dropout(0.2),  # Reduce Dropout
        Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001))),
        Dropout(0.2),
        LSTM(16, return_sequences=False, kernel_regularizer=l2(0.001)),  # Simplify the final LSTM layer
        Dropout(0.2),
        Dense(output_shape)  # Output layer
    ])
    model.compile(optimizer='RMSprop', loss='mean_squared_error')  # Use RMSprop optimizer
    return model

# Train and validate the LSTM model
def train_and_evaluate_lstm(X_train, y_train, X_test, y_test):
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]), y_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)  # Adjust early stopping
    model.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)
    return model

# Predict and calculate combined model results, adding weighted ensemble
def predict_combined_model(X_test, model_lstm, model_xgb, lstm_weight=0.5):
    y_pred_lstm = model_lstm.predict(X_test)
    y_pred_xgb = model_xgb.predict(X_test.reshape(X_test.shape[0], -1))
    return (lstm_weight * y_pred_lstm + (1 - lstm_weight) * y_pred_xgb)  # Weighted ensemble

# Read and process data
grouped_avg_properties_sa2_df = pd.read_csv('../data/curated/grouped_avg_properties_sa2.csv')
increase_rates_df = pd.read_csv('../data/curated/Updated_Rent_Data.csv')

# Find common SA2 areas and process
common_sa2_names = np.intersect1d(grouped_avg_properties_sa2_df['SA2_NAME21'], increase_rates_df['District'])
grouped_avg_properties_sa2_df = grouped_avg_properties_sa2_df[grouped_avg_properties_sa2_df['SA2_NAME21'].isin(common_sa2_names)].reset_index(drop=True)
increase_rates_df = increase_rates_df[increase_rates_df['District'].isin(common_sa2_names)].reset_index(drop=True)

# Extract rent data and external features
rent_columns = [col for col in increase_rates_df.columns if 'Rent' in col]
rent_data = increase_rates_df[rent_columns].values
external_features = grouped_avg_properties_sa2_df[['Distance (km)', 'School Distance (km)', 'Distance to Closest Shopping Center (km)', 'Price']].values

# Normalize the data
scaler_rent = MinMaxScaler()
scaler_features = MinMaxScaler()
rent_data_scaled = scaler_rent.fit_transform(rent_data)
external_features_scaled = scaler_features.fit_transform(external_features)

# Fill missing values and create time series data
rent_data_filled = pd.DataFrame(rent_data_scaled).fillna(method='bfill').fillna(method='ffill').values

# Create input data, including external features
sequence_length = 10  # Increase time window length
X, y = [], []
for i in range(sequence_length, rent_data_filled.shape[0]):
    X_sequence_rent = rent_data_filled[i-sequence_length:i]
    X_sequence_external = np.tile(external_features_scaled[i], (sequence_length, 1))
    X_sequence = np.concatenate((X_sequence_rent, X_sequence_external), axis=1)
    X.append(X_sequence)
    y.append(rent_data_filled[i])

X, y = np.array(X), np.array(y)

# Cross-validation and evaluation
tscv = TimeSeriesSplit(n_splits=5)
metrics_list = []

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train LSTM and XGBoost models
    model_lstm = train_and_evaluate_lstm(X_train, y_train, X_test, y_test)
    model_xgb = XGBRegressor(n_estimators=200, learning_rate=0.01)  # Reduce learning rate, increase number of trees
    model_xgb.fit(X_train.reshape(X_train.shape[0], -1), y_train)

    # Predict and evaluate
    y_pred_combined = predict_combined_model(X_test, model_lstm, model_xgb, lstm_weight=0.6)  # LSTM weighted 60%
    mae, mse, rmse, mape, r2 = calculate_metrics(y_test, y_pred_combined)
    metrics_list.append([mae, mse, rmse, mape, r2])
    print(f"Fold {fold + 1} - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, MAPE: {mape}%, R^2: {r2}")

# Output average evaluation metrics
metrics_avg = np.mean(metrics_list, axis=0)
print(f"Cross-validated MAE: {metrics_avg[0]}, MSE: {metrics_avg[1]}, RMSE: {metrics_avg[2]}, MAPE: {metrics_avg[3]}%, R^2: {metrics_avg[4]}")

# Final model training and future prediction
model_final_lstm = train_and_evaluate_lstm(X, y, X, y)
model_final_xgb = XGBRegressor(n_estimators=200, learning_rate=0.01)
model_final_xgb.fit(X.reshape(X.shape[0], -1), y)

# Use combined model to predict on the entire dataset
y_pred_final = predict_combined_model(X, model_final_lstm, model_final_xgb, lstm_weight=0.6)

# Inverse transform predictions and true values
y_pred_final = scaler_rent.inverse_transform(y_pred_final)
y_true_final = scaler_rent.inverse_transform(y)

# Calculate final evaluation metrics
mae_final, mse_final, rmse_final, mape_final, r2_final = calculate_metrics(y_true_final, y_pred_final)

# Output final evaluation metrics
print(f"Final model MAE: {mae_final}")
print(f"Final model MSE: {mse_final}")
print(f"Final model RMSE: {rmse_final}")
print(f"Final model MAPE: {mape_final}%")
print(f"Final model R-squared: {r2_final}")

# Generate future 12-quarter predictions
predicted_rent_prices_all = []
future_steps = 12
for i in range(len(grouped_avg_properties_sa2_df)):
    X_input_rent = rent_data_filled[-sequence_length:]
    X_input_external = np.tile(external_features_scaled[i], (sequence_length, 1))
    X_input = np.concatenate((X_input_rent, X_input_external), axis=1)
    X_input = np.expand_dims(X_input, axis=0)

    predicted_rent_sequence = []
    for step in range(future_steps):
        predicted_rent = model_final_lstm.predict(X_input)
        new_rent_data = predicted_rent.reshape(1, 1, -1)
        new_rent_with_features = np.concatenate((new_rent_data, np.tile(external_features_scaled[i], (1, 1, 1))), axis=2)
        X_input = np.concatenate([X_input[:, 1:, :], new_rent_with_features], axis=1)
        predicted_rent_sequence.append(scaler_rent.inverse_transform(predicted_rent)[0][0])

    predicted_rent_prices_all.append(np.array(predicted_rent_sequence).flatten()[:future_steps])

# Convert prediction results to the appropriate format
predicted_rent_prices_all = np.array(predicted_rent_prices_all)
future_dates = pd.date_range(start='2025-01-01', periods=future_steps, freq='Q').strftime('%Y-%m-%d')
predicted_rent_prices_df = pd.DataFrame(predicted_rent_prices_all, columns=future_dates)
predicted_rent_prices_df.insert(0, 'SA2_NAME', grouped_avg_properties_sa2_df['SA2_NAME21'])
predicted_rent_prices_df.to_csv('predicted_rent_prices_2025_2027_optimized_ensemble.csv', index=False)



2024-10-17 09:45:08.162880: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 09:45:08.169021: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 09:45:08.215754: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 09:45:08.258466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 09:45:08.299950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Fold 1 - MAE: 0.18362149918668785, MSE: 0.05206037065068047, RMSE: 0.22816741803044638, MAPE: 54.763460605297546%, R^2: -0.07028858273749618


/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Fold 2 - MAE: 0.13081745702109046, MSE: 0.024121632188888617, RMSE: 0.15531140392414403, MAPE: 66.46215174156441%, R^2: -0.9443612728570977


/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
Fold 5 - MAE: 0.1295812259189557, MSE: 0.03105207771809094, RMSE: 0.17621599733875168, MAPE: inf%, R^2: -0.06939932346540394
Cross-validated MAE: 0.11862747185973195, MSE: 0.02476709253216894, RMSE: 0.14806093972047105, MAPE: inf%, R^2: -0.6010263020925984


/tmp/ipykernel_742/1585066194.py:17: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Final model MAE: 32.99333149988511
Final model MSE: 2309.824122058716
Final model RMSE: 48.06062964692323
Final model MAPE: 7.205413060805134%
Final model R-squared: 0.552141731642741
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━

/tmp/ipykernel_742/1585066194.py:149: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_dates = pd.date_range(start='2025-01-01', periods=future_steps, freq='Q').strftime('%Y-%m-%d')


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, mape, r2

# Create and compile LSTM model, increasing complexity with adjusted Dropout and L2 regularization
def build_lstm_model(input_shape, output_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.001))),
        Dropout(0.3),  # Increased dropout to prevent overfitting
        Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))),
        Dropout(0.3),
        LSTM(32, return_sequences=False, kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        Dense(output_shape)  # Output layer
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')  # Using Adam optimizer
    return model

# Train and validate LSTM model, increasing patience and adjusting batch_size
def train_and_evaluate_lstm(X_train, y_train, X_test, y_test):
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]), y_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)  # Increased patience to avoid early stopping
    model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)
    return model

# Predict and compute combined model results with weighted fusion, reducing LSTM weight
def predict_combined_model(X_test, model_lstm, model_xgb, lstm_weight=0.4):  # Increase XGBoost weight, LSTM weight at 40%
    y_pred_lstm = model_lstm.predict(X_test)
    y_pred_xgb = model_xgb.predict(X_test.reshape(X_test.shape[0], -1))
    return (lstm_weight * y_pred_lstm + (1 - lstm_weight) * y_pred_xgb)

# Read and process data
grouped_avg_properties_sa2_df = pd.read_csv('../data/curated/grouped_avg_properties_sa2.csv')
increase_rates_df = pd.read_csv('../data/curated/Updated_Rent_Data.csv')

# Find common SA2 regions and process them
common_sa2_names = np.intersect1d(grouped_avg_properties_sa2_df['SA2_NAME21'], increase_rates_df['District'])
grouped_avg_properties_sa2_df = grouped_avg_properties_sa2_df[grouped_avg_properties_sa2_df['SA2_NAME21'].isin(common_sa2_names)].reset_index(drop=True)
increase_rates_df = increase_rates_df[increase_rates_df['District'].isin(common_sa2_names)].reset_index(drop=True)

# Extract rent data and external features
rent_columns = [col for col in increase_rates_df.columns if 'Rent' in col]
rent_data = increase_rates_df[rent_columns].values
external_features = grouped_avg_properties_sa2_df[['Distance (km)', 'School Distance (km)', 'Distance to Closest Shopping Center (km)', 'Price']].values

# Normalize the data
scaler_rent = MinMaxScaler()
scaler_features = MinMaxScaler()
rent_data_scaled = scaler_rent.fit_transform(rent_data)
external_features_scaled = scaler_features.fit_transform(external_features)

# Fill missing values and create time series data
rent_data_filled = pd.DataFrame(rent_data_scaled).fillna(method='bfill').fillna(method='ffill').values

# Create input data, including external features
sequence_length = 12  # Increase the sliding window length
X, y = [], []
for i in range(sequence_length, rent_data_filled.shape[0]):
    X_sequence_rent = rent_data_filled[i-sequence_length:i]
    X_sequence_external = np.tile(external_features_scaled[i], (sequence_length, 1))
    X_sequence = np.concatenate((X_sequence_rent, X_sequence_external), axis=1)
    X.append(X_sequence)
    y.append(rent_data_filled[i])

X, y = np.array(X), np.array(y)

# Cross-validation and evaluation
tscv = TimeSeriesSplit(n_splits=5)
metrics_list = []

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train LSTM and XGBoost models
    model_lstm = train_and_evaluate_lstm(X_train, y_train, X_test, y_test)
    
    # Adjust XGBoost parameters, increase number of trees, max_depth, and min_child_weight
    model_xgb = XGBRegressor(n_estimators=300, learning_rate=0.01, max_depth=5, min_child_weight=3)
    model_xgb.fit(X_train.reshape(X_train.shape[0], -1), y_train)

    # Predict and evaluate
    y_pred_combined = predict_combined_model(X_test, model_lstm, model_xgb, lstm_weight=0.4)  # LSTM weight reduced to 40%
    mae, mse, rmse, mape, r2 = calculate_metrics(y_test, y_pred_combined)
    metrics_list.append([mae, mse, rmse, mape, r2])
    print(f"Fold {fold + 1} - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, MAPE: {mape}%, R²: {r2}")

# Output average evaluation metrics
metrics_avg = np.mean(metrics_list, axis=0)
print(f"Cross-validated MAE: {metrics_avg[0]}, MSE: {metrics_avg[1]}, RMSE: {metrics_avg[2]}, MAPE: {metrics_avg[3]}%, R²: {metrics_avg[4]}")

# Final model training and future predictions
model_final_lstm = train_and_evaluate_lstm(X, y, X, y)
model_final_xgb = XGBRegressor(n_estimators=300, learning_rate=0.01, max_depth=5, min_child_weight=3)
model_final_xgb.fit(X.reshape(X.shape[0], -1), y)

# Use the combined model to predict on the entire dataset
y_pred_final = predict_combined_model(X, model_final_lstm, model_final_xgb, lstm_weight=0.4)

# Inverse transform predicted and true values
y_pred_final = scaler_rent.inverse_transform(y_pred_final)
y_true_final = scaler_rent.inverse_transform(y)

# Calculate final evaluation metrics
mae_final, mse_final, rmse_final, mape_final, r2_final = calculate_metrics(y_true_final, y_pred_final)

# Output final evaluation metrics
print(f"Final model MAE: {mae_final}")
print(f"Final model MSE: {mse_final}")
print(f"Final model RMSE: {rmse_final}")
print(f"Final model MAPE: {mape_final}%")
print(f"Final model R²: {r2_final}")

# Generate future 12-quarter predictions
predicted_rent_prices_all = []
future_steps = 12
for i in range(len(grouped_avg_properties_sa2_df)):
    X_input_rent = rent_data_filled[-sequence_length:]
    X_input_external = np.tile(external_features_scaled[i], (sequence_length, 1))
    X_input = np.concatenate((X_input_rent, X_input_external), axis=1)
    X_input = np.expand_dims(X_input, axis=0)

    predicted_rent_sequence = []
    for step in range(future_steps):
        predicted_rent = model_final_lstm.predict(X_input)
        new_rent_data = predicted_rent.reshape(1, 1, -1)
        new_rent_with_features = np.concatenate((new_rent_data, np.tile(external_features_scaled[i], (1, 1, 1))), axis=2)
        X_input = np.concatenate([X_input[:, 1:, :], new_rent_with_features], axis=1)
        predicted_rent_sequence.append(scaler_rent.inverse_transform(predicted_rent)[0][0])

    predicted_rent_prices_all.append(np.array(predicted_rent_sequence).flatten()[:future_steps])

# Convert predictions to the appropriate format
predicted_rent_prices_all = np.array(predicted_rent_prices_all)
future_dates = pd.date_range(start='2025-01-01', periods=future_steps, freq='Q').strftime('%Y-%m-%d')
predicted_rent_prices_df = pd.DataFrame(predicted_rent_prices_all, columns=future_dates)
predicted_rent_prices_df.insert(0, 'SA2_NAME', grouped_avg_properties_sa2_df['SA2_NAME21'])
predicted_rent_prices_df.to_csv('predicted_rent_prices_2025_2027_optimized_ensemble_2.csv', index=False)

# Output predicted results
print(predicted_rent_prices_df.head())


/tmp/ipykernel_742/3145224106.py:69: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rent_data_filled = pd.DataFrame(rent_data_scaled).fillna(method='bfill').fillna(method='ffill').values
/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
Fold 1 - MAE: 0.17548912053653673, MSE: 0.037187366801416315, RMSE: 0.192840262397188, MAPE: 69.06400958857235%, R²: -0.40616377110788604


/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Fold 2 - MAE: 0.10359050377083606, MSE: 0.01496067068532882, RMSE: 0.12231382050009239, MAPE: 47.82097236072104%, R²: -0.4587592420465119


/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Fold 5 - MAE: 0.14184177520475083, MSE: 0.03369331183090051, RMSE: 0.18355738021365556, MAPE: inf%, R²: -0.08044570899928243
Cross-validated MAE: 0.10751486996298901, MSE: 0.01934510892934773, RMSE: 0.12921072043357656, MAPE: inf%, R²: -0.24649683713830112


/tmp/ipykernel_742/3145224106.py:17: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
/home/xingyouf/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step
Final model MAE: 23.75514249870576
Final model MSE: 1239.7300916311874
Final model RMSE: 35.209801073439586
Final model MAPE: 5.305913322364315%
Final model R²: 0.7554234020429057
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━

/tmp/ipykernel_742/3145224106.py:151: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_dates = pd.date_range(start='2025-01-01', periods=future_steps, freq='Q').strftime('%Y-%m-%d')
